In [1]:
%matplotlib inline
%load_ext rpy2.ipython

In [2]:
%R require(ggplot2); require(GenometriCorr) ; require(rtracklayer);require( rtracklayer);\
require(IRanges)

/home/benjamin/anaconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: ggplot2

  warnings.warn(x, RRuntimeWarning)
/home/benjamin/anaconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: GenometriCorr

  warnings.warn(x, RRuntimeWarning)
/home/benjamin/anaconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: IRanges

  warnings.warn(x, RRuntimeWarning)
/home/benjamin/anaconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: BiocGenerics

  warnings.warn(x, RRuntimeWarning)
/home/benjamin/anaconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: parallel

  warnings.warn(x, RRuntimeWarning)
/home/benjamin/anaconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attaching pack

array([1], dtype=int32)

In [ ]:
import shutil, errno
from glob import glob as glb
import os

def copyanything(src, dest_dir):
    for filename in glb(os.path.join(src, '*.*')):
        shutil.copy(filename, dest_dir)

In [ ]:
from IPython.display import HTML
def PDF(url):
    return HTML('<iframe src=./tmp_figures/%s width=700 height=350></iframe>' % url)

In [ ]:
OUT_FOLDER_FIG = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/figures'
TMP_FIG_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/downstream_analysis_2017/scripts/tmp_figures'

In [ ]:
import pandas as pd 
# Make a pandas DataFrame
df = pd.DataFrame({'Alphabet': ['a', 'b', 'c', 'd','e', 'f', 'g', 'h','i'],
                   'A': [4, 3, 5, 2, 1, 7, 7, 5, 9],
                   'B': [0, 4, 3, 6, 7, 10,11, 9, 13],
                   'C': [1, 2, 3, 1, 2, 3, 1, 2, 3]})

In [ ]:
%%R -i df
ggplot(data=df) + geom_point(aes(x=A, y=B, color=C))

In [ ]:
%%R 
library("rtracklayer")
library("TxDb.Hsapiens.UCSC.hg19.knownGene")
library("IRanges")
refseq <- transcripts(TxDb.Hsapiens.UCSC.hg19.knownGene)
cpgis <- import(system.file("extdata", "UCSCcpgis_hg19.bed", package = "GenometriCorr"))
seqinfo(cpgis) <- seqinfo(TxDb.Hsapiens.UCSC.hg19.knownGene)[seqnames(seqinfo(cpgis))]
cpgis
GenometriCorr:::VisualiseTwoIRanges(ranges(cpgis[seqnames(cpgis) == 'chr1']), ranges(refseq[seqnames(refseq) == "chr1"]), nameA = "CpG Islands", nameB = "RefSeq Genes", chrom_length = seqlengths(TxDb.Hsapiens.UCSC.hg19.knownGene)["chr1"],title = "CpGIslands and RefGenes on chr1 of Hg19")

In [ ]:
%%R
cpgis <- import(system.file("extdata", "UCSCcpgis_hg19.bed", package = "GenometriCorr"))
seqinfo(cpgis) <- seqinfo(TxDb.Hsapiens.UCSC.hg19.knownGene)[seqnames(seqinfo(cpgis))]
cpgis

In [ ]:
%%R
pn.area <- 100
pn.dist <- 100
pn.jacc <- 100
cpgi_to_genes <- GenometriCorr:::GenometriCorrelation(cpgis, refseq, chromosomes.to.proceed = c("chr1","chr2", "chr3"), ecdf.area.permut.number = pn.area,mean.distance.permut.number = pn.dist,jaccard.measure.permut.number = pn.jacc,keep.distributions = TRUE, showProgressBar = FALSE)
print(cpgi_to_genes)

In [ ]:
%%R
GenometriCorr:::graphical.report(cpgi_to_genes, pdffile = "CpGi_to_RefSeq_chr1_picture.pdf", show.chromosomes = c("chr1"), show.all = FALSE)

In [ ]:
%%R
setwd("/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/lists")
getwd()
busco_bed <- import("Pst_104E_v12_p_busco.gene.bed", format ='bed')
effector_bed <- import("Pst_104E_v12_p_effector.gene.bed", format='bed')
no_effector_bed <- import("Pst_104E_v12_p_effectorp_noeffector.gene.bed", format='bed')
#Pst_104E_seqinfo <- import("../../../032017_assembly/Pst_104E_v12_p_ctg.genome_file", as = "Rle")
#Pst_104E_seqinfo
#effector_bed
pn.area <- 100
pn.dist <- 100
pn.jacc <- 100
effector_to_busco <- GenometriCorr:::GenometriCorrelation(effector_bed, busco_bed,  ecdf.area.permut.number = pn.area,mean.distance.permut.number = pn.dist,jaccard.measure.permut.number = pn.jacc,keep.distributions = TRUE, showProgressBar = FALSE)
effector_to_no_effector <- GenometriCorr:::GenometriCorrelation(effector_bed, no_effector_bed,  ecdf.area.permut.number = pn.area,mean.distance.permut.number = pn.dist,jaccard.measure.permut.number = pn.jacc,keep.distributions = TRUE, showProgressBar = FALSE)


In [ ]:
%%R
GenometriCorr:::graphical.report(effector_to_busco, pdffile = "../figures/Pst_104E_p_effector_to_busco.pdf",  show.all = TRUE)
GenometriCorr:::graphical.report(effector_to_no_effector , pdffile = "../figures/Pst_104E_p_effector_to_no_effector.pdf",  show.all = TRUE)
GenometriCorr:::visualize(effector_to_busco, pdffile = "../figures/Pst_104E_p_effector_to_busco_vis.pdf",show.all = TRUE)
GenometriCorr:::visualize(effector_to_no_effector, pdffile = "../figures/Pst_104E_p_effector_to_no_effector_vis.pdf",show.all = TRUE)


In [ ]:
%%R
busco_bed_to_no_effector <- GenometriCorr:::GenometriCorrelation(busco_bed, no_effector_bed,  ecdf.area.permut.number = pn.area,mean.distance.permut.number = pn.dist,jaccard.measure.permut.number = pn.jacc,keep.distributions = TRUE, showProgressBar = FALSE)
GenometriCorr:::graphical.report(busco_bed_to_no_effector , pdffile = "../figures/Pst_104E_p_busco_to_no_effector.pdf",  show.all = TRUE)
GenometriCorr:::visualize(busco_bed_to_no_effector, pdffile = "../figures/Pst_104E_p_busco_to_no_effector_vis.pdf",show.all = TRUE)

In [ ]:
!ls

In [ ]:
%%R
#no_busco_bed <- import("Pst_104E_v12_p_non_busco.gene.bed", format ='bed')
#busco_bed_to_no_busco <- GenometriCorr:::GenometriCorrelation(no_busco_bed , no_effector_bed,  ecdf.area.permut.number = pn.area,mean.distance.permut.number = pn.dist,jaccard.measure.permut.number = pn.jacc,keep.distributions = TRUE, showProgressBar = FALSE)
GenometriCorr:::graphical.report(busco_bed_to_no_busco , pdffile = "../figures/Pst_104E_p_busco_to_no_busco.pdf",  show.all = TRUE)
GenometriCorr:::visualize(busco_bed_to_no_busco, pdffile = "../figures/Pst_104E_p_busco_to_no_busco_vis.pdf",show.all = TRUE)

Doing comparison with TEs

In [ ]:
%%R
#now compare relations ship between TEs greater 400 and all sets of genes
setwd("/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/lists")
getwd()
busco_bed <- import("Pst_104E_v12_p_busco.gene.bed", format ='bed')
effector_bed <- import("Pst_104E_v12_p_effector.gene.bed", format='bed')
no_effector_bed <- import("Pst_104E_v12_p_effectorp_noeffector.gene.bed", format='bed')
TE_greater_400_bed <- import("Pst_104E_v12_p_ctg.REPET.sorted.greater400.superfamily.gff")
TE_g400_to_busco <- GenometriCorr:::GenometriCorrelation(TE_greater_400_bed, busco_bed,  ecdf.area.permut.number = pn.area,mean.distance.permut.number = pn.dist,jaccard.measure.permut.number = pn.jacc,keep.distributions = TRUE, showProgressBar = FALSE)
TE_g400_to_effector <- GenometriCorr:::GenometriCorrelation(TE_greater_400_bed, effector_bed,  ecdf.area.permut.number = pn.area,mean.distance.permut.number = pn.dist,jaccard.measure.permut.number = pn.jacc,keep.distributions = TRUE, showProgressBar = FALSE)
TE_g400_to_no_effector <- GenometriCorr:::GenometriCorrelation(TE_greater_400_bed, no_effector_bed,  ecdf.area.permut.number = pn.area,mean.distance.permut.number = pn.dist,jaccard.measure.permut.number = pn.jacc,keep.distributions = TRUE, showProgressBar = FALSE)

In [ ]:
%%R

GenometriCorr:::graphical.report(TE_g400_to_busco  , pdffile = "../figures/Pst_104E_p_TE_g400_to_busco.pdf",  show.all = TRUE)
GenometriCorr:::visualize(TE_g400_to_busco , pdffile = "../figures/Pst_104E_p_busco_TE_g400_to_busco_vis.pdf",show.all = TRUE)

GenometriCorr:::graphical.report(TE_g400_to_effector  , pdffile = "../figures/Pst_104E_p_TE_g400_to_effector.pdf",  show.all = TRUE)
GenometriCorr:::visualize(TE_g400_to_effector , pdffile = "../figures/Pst_104E_p_busco_TE_g400_to_effector_vis.pdf",show.all = TRUE)

GenometriCorr:::graphical.report(TE_g400_to_no_effector  , pdffile = "../figures/Pst_104E_p_TE_g400_to_no_effector.pdf",  show.all = TRUE)
GenometriCorr:::visualize(TE_g400_to_no_effector , pdffile = "../figures/Pst_104E_p_busco_TE_g400_to_no_effector.pdf",show.all = TRUE)

In [ ]:
#this moves the file to a folder were jupyter can access it for displaying.
copyanything(OUT_FOLDER_FIG, TMP_FIG_PATH)

In [ ]:
PDF('Pst_104E_p_effector_to_no_effector.pdf')

In [ ]:
PDF('Pst_104E_p_effector_to_busco_vis.pdf')

In [ ]:
PDF("Pst_104E_p_TE_g400_to_busco.pdf")

In [ ]:
PDF("Pst_104E_p_TE_g400_to_effector.pdf")

In [ ]:
PDF("Pst_104E_p_TE_g400_to_no_effector.pdf")

In [ ]:
PDF('Pst_104E_p_busco_to_no_busco.pdf')

In [ ]:
#not working as this also includes buscos. Needs to be none BUSCOS
PDF('Pst_104E_p_busco_to_no_effector.pdf')